In [ ]:
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

sns.set_style("whitegrid")

efficiency_csp_power_block = 0.412  # from data/efficiencies.csv

nice_names = {
    "csp-tower": "CSP",
    "csp-tower power block": "CSP",
    "solar-utility": "PV",
    "onwind": "Wind onshore",
    "offwind": "Wind offshore",
}

nice_colors = {
    "CSP": "orange",
    "PV": "#f9d002",
    "Wind onshore": "#235ebc",
    "Wind offshore": "#6895dd",
}

# Select data to plot
df = pd.read_csv(snakemake.input["results"], delimiter=";")
df = df.loc[
    (df["exporter"] == snakemake.wildcards["exporter"])
    & (df["esc"] == snakemake.wildcards["esc"])
]

df["flexibility"] = df["scenario"].str.replace("_.*", "", regex=True)
df["csp"] = df["scenario"].str.contains("with_csp")

# Add sorted category column to df indicating type of flexibility
# then sort by that category to ensure correct order in x-axis of plot
# Note: Reproduces the order given in "flexibilities".

flexibilities = {
    "unbuffered": "No flexibility",
    "daily": "Daily",
    "weekly": "Weekly",
    "biweekly": "Biweekly",
    "monthly": "Month",
    "quaterly": "Quaterly",
    "annually": "Yearly",
}

df["flexibility"] = df.flexibility.replace(flexibilities)

df["flexibility"] = pd.Categorical(
    df.flexibility, categories=flexibilities.values(), ordered=True
)

df = df.sort_values("flexibility")

# Restrict to only needed information from now on
df = df.query("category == 'RES'")
df = df.drop(
    columns=["year", "wacc", "esc", "exporter", "importer", "category", "scenario"]
)

technologies = [
    "onwind",
    "offwind",
    "solar-utility",
    "csp-tower power block",
    "csp-tower",
]

electricity_columns = [f"Total generated electricity from {t}" for t in technologies]
capacity_columns = [f"Total installed capacity {t}" for t in technologies]

electricity = df.loc[df["subcategory"].isin(electricity_columns)]
electricity["subcategory"] = electricity["subcategory"].str.replace(
    "Total generated electricity from ", ""
)
electricity = electricity.pivot_table(
    index=["csp", "flexibility"], columns="subcategory", values="value"
)

capacity = df.loc[df["subcategory"].isin(capacity_columns)]
capacity["subcategory"] = capacity["subcategory"].str.replace(
    "Total installed capacity ", ""
)
capacity = capacity.pivot_table(
    index=["csp", "flexibility"], columns="subcategory", values="value"
)
capacity[
    "csp-tower"
] *= efficiency_csp_power_block  # Weigh CSP capacity with heat-to-electricity efficiency of CSP


electricity /= 1e6  # in TWh
capacity /= 1e3  # in GW

# Rename to nicer names for plot legend
electricity = electricity.rename(
    columns={c: nice_names[c] for c in electricity.columns}
)
capacity = capacity.rename(columns={c: nice_names[c] for c in capacity.columns})

# Reorder for alphabetical legend order
electricity = electricity[sorted(electricity.columns)]
capacity = capacity[sorted(capacity.columns)]

with open(snakemake.log[0], "w") as f:
    print(
        "Capacity difference in GW (highest/lowest capacity of all flexibilities)",
        file=f,
    )
    print(
        capacity.sum(axis="columns")
        .groupby("csp")
        .agg([np.ptp, lambda x: np.round(np.ptp(x) / np.max(x), 4) * 100]),
        file=f,
    )

technologies = ["Wind offshore", "Wind onshore", "PV", "CSP"]
wo_csp_columns = [s + " (w/o CSP)" for s in technologies]
w_csp_columns = [s + " (w/ CSP)" for s in technologies]

## Plotting

fig, axes = plt.subplots(2, 1, figsize=(7, 5), sharex=True)
# Upper plot for generation capacities
ax = axes[0]

# Construct dataframe with columns for all techs w/ and w/o CSP using auxiliary df
df = capacity.reset_index().set_index("flexibility")
df_wo_csp = df.query("csp == False")
df_w_csp = df.query("csp == True")
df_wo_csp = df_wo_csp.rename(
    columns={k: v for k, v in zip(technologies, wo_csp_columns)}
)
df_w_csp = df_w_csp.rename(columns={k: v for k, v in zip(technologies, w_csp_columns)})
df_w_csp = df_w_csp.drop(columns="csp")
df_wo_csp = df_wo_csp.drop(columns="csp")
df = df_w_csp.join(df_wo_csp)

df[wo_csp_columns].plot.bar(
    position=1,
    width=0.3,
    stacked=True,
    color=[nice_colors[t] for t in technologies],
    ax=ax,
)
df[w_csp_columns].plot.bar(
    position=0,
    width=0.3,
    stacked=True,
    color=[nice_colors[t] for t in technologies],
    hatch="///",
    ax=ax,
)

## Edit legend

# For the two headings, add two empty handles
empty_handles = [ax.plot([], marker="", ls="")[0]] * 2

# Labels without "w/ CSP"|"w/o CSP" addition
# Reorder handles for 2x4 column layout and add the empty helper from above
# such that CSP for the "No CSP" scenario does not show up and only
# the hatched label icon is there for CSP
new_handles = ax.get_legend_handles_labels()[0]
new_handles = (
    empty_handles[:1] * 2  # So that CSP for "w/o CSP" scenario has no handle
    + new_handles[:3][::-1]
    + empty_handles[1:]
    + new_handles[4:][::-1]
)
new_labels = ["w/o"] + [""] * 4 + ["w/     CSP"] + technologies[::-1]
legend = ax.legend(
    handles=new_handles,
    labels=new_labels,
    fontsize="small",
    ncol=2,
    columnspacing=0.0,
    bbox_to_anchor=(1, 0),
    loc="lower left",
)
# Remove spacing such that column headings are above color handles
for vpack in legend._legend_handle_box.get_children():
    for hpack in vpack.get_children()[:1]:
        hpack.get_children()[0].set_width(-5)

ax.set_xlabel("Flexibility to shift load")
ax.set_ylabel("Installed Capacity [GW]")
ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
ax.tick_params(axis="both", which="major", labelsize="small")

ax.set_ylim(0, 89)
ax.set_xlim(
    -0.5,
)

# Lower plot for annual generation volume
ax = axes[1]

# Construct dataframe with columns for all techs w/ and w/o CSP using auxiliary df
df = electricity.reset_index().set_index("flexibility")
df_wo_csp = df.query("csp == False")
df_w_csp = df.query("csp == True")
df_wo_csp = df_wo_csp.rename(
    columns={k: v for k, v in zip(technologies, wo_csp_columns)}
)
df_w_csp = df_w_csp.rename(columns={k: v for k, v in zip(technologies, w_csp_columns)})
df_w_csp = df_w_csp.drop(columns="csp")
df_wo_csp = df_wo_csp.drop(columns="csp")
df = df_w_csp.join(df_wo_csp)

df[wo_csp_columns].plot.bar(
    position=1,
    width=0.3,
    stacked=True,
    color=[nice_colors[t] for t in technologies],
    ax=ax,
)
df[w_csp_columns].plot.bar(
    position=0,
    width=0.3,
    stacked=True,
    color=[nice_colors[t] for t in technologies],
    hatch="///",
    ax=ax,
)

ax.set_xlabel("Flexibility to shift load")
ax.set_ylabel("Generation [TWh]")

ax.set_xticklabels(ax.get_xticklabels(), rotation=0, fontsize="small")
ax.tick_params(axis="both", which="major", labelsize="small")

# Legend only in upper plot
ax.legend().remove()

ax.set_ylim(0, 89)
ax.set_xlim(
    -0.5,
)

fig.tight_layout(pad=0)

# Saving figure
fig.savefig(snakemake.output["pdf"], dpi=300, bbox_inches="tight")
fig.savefig(snakemake.output["png"], dpi=300, bbox_inches="tight")